#### 0.数据探索部分
这个笔记本展示了如何对./data/raw中的原始数据集进行预处理，最终生成可供建模的数据集，保存在./data/processed中。

In [1]:
import pandas as pd
ProjectFolderDir = '..'   # 项目文件夹的路径是相对于本笔记本路径而言的，当然也可以使用绝对路径

初步分析阶段先将字段以字符串形式录入，便于后期统一格式

In [2]:
BigData = pd.read_csv(ProjectFolderDir + "/data/raw/浙江大学-图书外借数据-2013~2018.csv", encoding='utf8', dtype=str)
BigData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2067948 entries, 0 to 2067947
Data columns (total 27 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   UNIVERSITY_ID      object
 1   ITEM_ID            object
 2   SUBLIBRARY         object
 3   LOAN_DATE          object
 4   LOAN_HOUR          object
 5   DUE_DATE           object
 6   DUE_HOUR           object
 7   RETURNED_DATE      object
 8   RETURNED_HOUR      object
 9   RETURNED_LOCATION  object
 10  ITEM_STATUS        object
 11  RENEWAL_NO         object
 12  LASTRENEW_DATE     object
 13  RECALL_DATE        object
 14  RECALL_DUE_DATE    object
 15  ITEM_CALLNO        object
 16  PUBLISH_YEAR       object
 17  AUTHOR             object
 18  TITLE              object
 19  PRESS              object
 20  ISBN               object
 21  HOLD_DAYS          object
 22  OVERDUE_DAYS       object
 23  PATRON_ID          object
 24  STUDENT_GRADE      object
 25  PATRON_DEPT        object
 26  PATRON_TYPE   

In [3]:
SmallData = pd.read_csv(ProjectFolderDir + "/data/raw/浙江大学-图书外借数据-2019.csv", encoding='utf8', dtype=str)
SmallData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192701 entries, 0 to 192700
Data columns (total 26 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   UNIVERSITY_ID    192701 non-null  object
 1   ITEM_ID          192701 non-null  object
 2   SUBLIBRARY       192701 non-null  object
 3   LOAN_DATE        192701 non-null  object
 4   LOAN_HOUR        192701 non-null  object
 5   DUE_DATE         192701 non-null  object
 6   DUE_HOUR         192701 non-null  object
 7   RETURNED_DATE    192701 non-null  object
 8   RETURNED_HOUR    192701 non-null  object
 9   ITEM_STATUS      192701 non-null  object
 10  RENEWAL_NO       192701 non-null  object
 11  LASTRENEW_DATE   192701 non-null  object
 12  RECALL_DATE      192701 non-null  object
 13  RECALL_DUE_DATE  192701 non-null  object
 14  ITEM_CALLNO      192696 non-null  object
 15  PUBLISH_YEAR     192701 non-null  object
 16  AUTHOR           192409 non-null  object
 17  TITLE     

In [4]:
BigData.sample(3)

,UNIVERSITY_ID,ITEM_ID,SUBLIBRARY,LOAN_DATE,LOAN_HOUR,DUE_DATE,DUE_HOUR,RETURNED_DATE,RETURNED_HOUR,RETURNED_LOCATION,...,AUTHOR,TITLE,PRESS,ISBN,HOLD_DAYS,OVERDUE_DAYS,PATRON_ID,STUDENT_GRADE,PATRON_DEPT,PATRON_TYPE
1113286,10335,000118316000050,BLT,20130110,1806,20130228,2359,20130121,1053,NaN,...,潘鼎坤,高等数学内容提要及解题指导 :少学时类,西安交通大学出版社,7-5605-1837-0,11,-38,0d70fcf70100be633ff55219aeb6095b,2012,传媒与国际文化学院,全日制本科生
846796,10335,000734009000100,BHB,20131025,1211,20140111,2359,20131215,1218,NaN,...,博赞,快速阅读,中信出版社,978-7-5086-1503-5,51,-27,724e62f1083c5d00f355ff25893b9e58,2012,管理学院,全日制本科生
1337704,10335,000689819000010,TLT,20151116,1326,20151226,2359,20151223,1303,NaN,...,王承学,化学反应工程,化学工业出版社,978-7-122-03645-2,37,-3,da74cf9b887910eec7fdbef6349ef577,2015,化学工程与生物工程学院,全日制硕士生


下面是构建模型所需要的数据列

In [5]:
user_columns = ['PATRON_ID', 'STUDENT_GRADE', 'PATRON_DEPT', 'PATRON_TYPE']
# 用户侧特征       用户id          年级            学生学院       学生类型
item_columns = ['ITEM_ID', 'SUBLIBRARY', 'ITEM_CALLNO', 'PUBLISH_YEAR', 'AUTHOR', 'TITLE', 'PRESS']
# 物品侧特征       记录号      馆藏地         图书索书号        出版年          作者       题目    出版社
time = 'LOAN_DATE'

将两份数据集合并

In [6]:
UsedColumns = user_columns + item_columns + [time]
UsedData = pd.concat([BigData[UsedColumns], SmallData[UsedColumns]])

In [7]:
UsedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2260649 entries, 0 to 192700
Data columns (total 12 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   PATRON_ID      object
 1   STUDENT_GRADE  object
 2   PATRON_DEPT    object
 3   PATRON_TYPE    object
 4   ITEM_ID        object
 5   SUBLIBRARY     object
 6   ITEM_CALLNO    object
 7   PUBLISH_YEAR   object
 8   AUTHOR         object
 9   TITLE          object
 10  PRESS          object
 11  LOAN_DATE      object
dtypes: object(12)
memory usage: 224.2+ MB


In [8]:
UsedData.sample(3)

,PATRON_ID,STUDENT_GRADE,PATRON_DEPT,PATRON_TYPE,ITEM_ID,SUBLIBRARY,ITEM_CALLNO,PUBLISH_YEAR,AUTHOR,TITLE,PRESS,LOAN_DATE
139216,5522d2e92156b97a1a72f9f6266fe539,2011,人文学院,全日制博士生,000464235000030,SLT,I206.7/CC9.3,2004,"陈思和,",无名时代的文学批评,广西师范大学出版社,20160325
387560,482d341523e161b4d85e3ccc6213d4eb,2010,传媒与国际文化学院,全日制本科生,000296869000010,SLT,I247.5/BZ11.1,1985,张辛欣,张辛欣小说集,北方文艺出版社,20140507
1340664,6ad1a59195c168ed291b78d8f915fb5e,2015,医学院,全日制本科生,000103384000020,MLT,R329/CZ2-5/f,2003,刘少纯,组织学与胚胎学学习与解题指南,华中科技大学出版社,20170109


本项目将借阅记录中的空值字段采用相同值’na'进行填充，特征处理时将其作为空特征。对于图书类别，将其索书号按/分割后得到图书大类CALLNO1与小类CALLNO2，将借阅日期转为整形，便于之后比较大小。

In [9]:
import re

def get_Bletter(str0):   # 取出大写字母
    b = re.sub(u"([^\u0041-\u007a])", "", str0)
    return b

UsedData = UsedData.fillna(value='na')
UsedData['CALLNO1'] = UsedData['ITEM_CALLNO'].str.split('/', expand=True)[0].map(lambda x: get_Bletter(str(x)))
UsedData['CALLNO2'] = UsedData['ITEM_CALLNO'].str.split('/', expand=True)[1].map(lambda x: get_Bletter(str(x)))
UsedData[time]=UsedData[time].astype(int)
UsedData = UsedData.drop(columns='ITEM_CALLNO')  # 删掉不再需要的列x

此时可以看到UsedData中的空值消失，并且多出了两个类别列

In [10]:
UsedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2260649 entries, 0 to 192700
Data columns (total 13 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   PATRON_ID      object
 1   STUDENT_GRADE  object
 2   PATRON_DEPT    object
 3   PATRON_TYPE    object
 4   ITEM_ID        object
 5   SUBLIBRARY     object
 6   PUBLISH_YEAR   object
 7   AUTHOR         object
 8   TITLE          object
 9   PRESS          object
 10  LOAN_DATE      int64 
 11  CALLNO1        object
 12  CALLNO2        object
dtypes: int64(1), object(12)
memory usage: 241.5+ MB


将数据存入./data/processed中，作为后续分析数据

In [11]:
import os

UsedData.to_csv(ProjectFolderDir + "/data/processed/ZJULibrary2013_2019.csv")

可以再读取下来观察一下是否与之前的处理结果一样

In [15]:
Data = pd.read_csv(ProjectFolderDir + "/data/processed/ZJULibrary2013_2019.csv", index_col = 0, dtype= str)
Data[time]=Data[time].astype(int)
Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2260649 entries, 0 to 192700
Data columns (total 13 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   PATRON_ID      object
 1   STUDENT_GRADE  object
 2   PATRON_DEPT    object
 3   PATRON_TYPE    object
 4   ITEM_ID        object
 5   SUBLIBRARY     object
 6   PUBLISH_YEAR   object
 7   AUTHOR         object
 8   TITLE          object
 9   PRESS          object
 10  LOAN_DATE      int64 
 11  CALLNO1        object
 12  CALLNO2        object
dtypes: int64(1), object(12)
memory usage: 241.5+ MB


至此完成了原始数据集的合并，填补了数据集中的空值，得到了图书的类别数据，最后将数据重新写入./data/processed/ZJULibrary2013_2019.csv中。

现在已经初步处理好了需要的全量数据，现在通过scikit-learn中的一些库函数对于借阅记录进行特征编码等，最终生成的numpy可读文件data_process.npy包含(保存在./data/processed/data_process.npy中)

1.所有物品的原始特征集item_set

2.所有用户的原始特征集user_set

3.训练集/验证集/测试集 特征向量 x_train/x_val/x_test

4.训练集/验证集/测试集 标签 y_train/y_val/y_test

In [ ]:
from sklearn.preprocessing import LabelEncoder

user_features = ['PATRON_ID', 'STUDENT_GRADE', 'PATRON_DEPT', 'PATRON_TYPE']
# 用户侧特征       用户id          年级            学生学院       学生类型
item_features = ['ITEM_ID', 'SUBLIBRARY', 'CALLNO1','CALLNO2', 'PUBLISH_YEAR', 'AUTHOR', 'TITLE', 'PRESS']
# 物品侧特征       记录号      馆藏地         大类1      小类2       出版年          作者       题目    出版社
features = user_features + item_features

feature_max_idx = {}    # 记录每个特征共有多少种
for feature in features:
    lbe = LabelEncoder()
    Data[feature] = lbe.fit_transform(Data[feature]) + 1  # 从1开始
    feature_max_idx[feature] = Data[feature].max() + 1 # 防止padding后造成分类标签越界
    if feature == 'PATRON_ID':
        user_map = {encode_id + 1: raw_id for encode_id, raw_id in enumerate(lbe.classes_)}
    if feature == 'ITEM_ID':
        item_map = {encode_id + 1: raw_id for encode_id, raw_id in enumerate(lbe.classes_)}

In [ ]:
Data.sample(3)

In [ ]:
import numpy as np
import os
np.save(os.path.join(ProjectFolderDir + '/data/processed/', "raw_id_maps.npy") ,np.array((user_map, item_map), dtype=object))

运行上方代码后会在./data/processed文件夹下生成储存数据代号标签与原数据的字典集raw_id_maps.npy

In [ ]:
feature_max_idx  # 可以观察每一个字段有几类不同值，统计用户人数与图书本树